In [1]:
# A couple of ways to import and slice-dice the fixture list using the premier league API

#imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import xlwings as xw
from dateutil import tz
import pytz

In [2]:
#pick up the fixture list and the static data (fixtures uses team ids) from the api. Basically lots of JSON files 
url = 'https://fantasy.premierleague.com/api/fixtures/'
r = requests.get(url)
fixtures_json = r.json()

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
static_json = r.json()


In [3]:
#from the team static set up a dictionary of team id to team name
teams = static_json["teams"]
team_codes ={team["id"]:team["name"] for team in teams}


In [4]:
#Loop through the fixtures and put in a dataframe. Can't parse the date as it has nulls which causes problems here
fixture_list = [[team_codes[fixture['team_h']],team_codes[fixture['team_a']],fixture['kickoff_time']] for fixture in fixtures_json]


In [5]:
# Attempt 1 - filter the list above by team

#define the team you want
team = "Liverpool"

#tidy up the dataframe and parse the dates (works beter outside the loop)    
dfFixtures = pd.DataFrame(fixture_list,columns=["Home","Away","Date/Time"])
dfFixtures["Date/Time"] = pd.to_datetime(dfFixtures["Date/Time"],format="%Y-%m-%dT%H:%M:%S",utc=True)
#map to local time
dfFixtures["Date/Time"]= dfFixtures["Date/Time"][2:].map(lambda x :x.astimezone(tz.tzlocal()))

teamFixtures = dfFixtures[(dfFixtures["Home"]==team) | (dfFixtures["Away"]==team)]

display(teamFixtures.style.hide_index())

### output to excel. commented out.
#sht = xw.Book().sheets[0]
#sht.range('A1').options(index=False).value = teamFixtures





Home,Away,Date/Time
Liverpool,Leeds,2020-09-12 17:30:00+01:00
Chelsea,Liverpool,2020-09-20 16:30:00+01:00
Liverpool,Arsenal,2020-09-28 20:00:00+01:00
Aston Villa,Liverpool,2020-10-04 19:15:00+01:00
Everton,Liverpool,2020-10-17 15:00:00+01:00
Liverpool,Sheffield Utd,2020-10-24 15:00:00+01:00
Liverpool,West Ham,2020-10-31 15:00:00+00:00
Man City,Liverpool,2020-11-07 15:00:00+00:00
Liverpool,Leicester,2020-11-21 15:00:00+00:00
Brighton,Liverpool,2020-11-28 15:00:00+00:00


In [37]:
# Attempt 2 - create a custom output format and iterate over the fixture list 
#define the team you want

utc = pytz.utc
#first define the function for the custom output text
def format_fixture(x,team):
    home=x[0]
    away=x[1]
    date = utc.localize(dt.datetime.strptime(x[2],"%Y-%m-%dT%H:%M:%SZ")).astimezone(tz.tzlocal())
    if home ==team:
        return ["{}".format(dt.datetime.strftime(date,"%d-%b-%y"))," {}(H) {}".format(away,dt.datetime.strftime(date,"%H:%M"))]
    elif away ==team:
        return ["{}".format(dt.datetime.strftime(date,"%d-%b-%y"))," {}(A) {}".format(home,dt.datetime.strftime(date,"%H:%M"))]

#conditonal formatting function    
def home_bold(val):
    bold = 'bold' if "(H)" in val  else ''
    return 'font-weight:{}'.format(bold)
    
#then put through some pythonic loops to get a dataframe of date and fixture 
def print_fixtures(team):
    output = [format_fixture(x,team) for x in fixture_list[2:] if format_fixture(x,team) != None]

    #print ditching the index and using contitional format
    df = pd.DataFrame(output,columns=["Date","Fixture"])
    return df

In [38]:
sht = xw.Book().sheets[0]
sht.range('d1').options(index=False).value = print_fixtures("Spurs")
sht.range('a1').options(index=False).value = print_fixtures("Liverpool")

In [39]:
print(print_fixtures("Liverpool"))

         Date                   Fixture
0   12-Sep-20            Leeds(H) 17:30
1   20-Sep-20          Chelsea(A) 16:30
2   28-Sep-20          Arsenal(H) 20:00
3   04-Oct-20      Aston Villa(A) 19:15
4   17-Oct-20          Everton(A) 15:00
5   24-Oct-20    Sheffield Utd(H) 15:00
6   31-Oct-20         West Ham(H) 15:00
7   07-Nov-20         Man City(A) 15:00
8   21-Nov-20        Leicester(H) 15:00
9   28-Nov-20         Brighton(A) 15:00
10  05-Dec-20           Wolves(H) 15:00
11  12-Dec-20           Fulham(A) 15:00
12  16-Dec-20            Spurs(H) 20:00
13  19-Dec-20   Crystal Palace(A) 15:00
14  26-Dec-20        West Brom(H) 15:00
15  28-Dec-20        Newcastle(A) 15:00
16  02-Jan-21      Southampton(A) 15:00
17  13-Jan-21          Burnley(H) 20:00
18  16-Jan-21          Man Utd(H) 15:00
19  27-Jan-21            Spurs(A) 19:45
20  30-Jan-21         West Ham(A) 15:00
21  03-Feb-21         Brighton(H) 20:00
22  06-Feb-21         Man City(H) 15:00
23  13-Feb-21        Leicester(A) 15:00


In [9]:
# getting the timezone right
x = dt.datetime.utcnow()
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))
utc = pytz.utc
x= utc.localize(x)
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))
x = x.astimezone(tz.tzlocal())
print(x.strftime('%m/%d/%Y %H:%M:%S %Z'))

09/25/2020 07:24:34 
09/25/2020 07:24:34 UTC
09/25/2020 08:24:34 GMT Daylight Time
